## Load data

In [90]:
library(jsonlite)
library(metafor)
library(ggplot2)
library(dplyr)

df <- fromJSON("/mnt/d/PYDataScience/HIV_TXP_SR/data/supp.json")
# Use the dim function to get the shape of the DataFrame
shape <- dim(df)

# Printing the shape
print(paste("Number of rows:", shape[1]))
print(paste("Number of columns:", shape[2]))

# Use the colnames or names function to get the column names
column_names <- colnames(df)
# Print the column names
print(column_names)

[1] "Number of rows: 48"
[1] "Number of columns: 114"
  [1] "study_id"                                                  
  [2] "author"                                                    
  [3] "year_of_publication"                                       
  [4] "country_of_study"                                          
  [5] "study_type"                                                
  [6] "sample_size_(hiv_seropositive_only)"                       
  [7] "intervention"                                              
  [8] "outcome"                                                   
  [9] "conclusion"                                                
 [10] "hiv_negative_samples"                                      
 [11] "intervention.1"                                            
 [12] "outcome.1"                                                 
 [13] "conclusion.1"                                              
 [14] "remarks"                                                   
 [15] "i

## Subgroup analysis
### Meta-regression for different variables on kidney transplant outcome

In [160]:
# Function to convert logit to proportion
logit_to_proportion <- function(logit) {
  exp(logit) / (1 + exp(logit))
}

meta_regression_function <- function(df, study_id_col, sample_size_col, effect_measure_col) {
  results <- list()

  # List of columns to perform meta-regression on
  columns_to_analyze <- setdiff(names(df), c(study_id_col, sample_size_col, effect_measure_col))

  for (col in columns_to_analyze) {
    # Prepare data for meta-regression
    mod_data <- df[, c(sample_size_col, effect_measure_col, col)]
    mod_data <- na.omit(mod_data)
    names(mod_data) <- c("sample_size", "effect_measure", "moderator")

    if (dim(mod_data)[1] < 2) {
      next
    }

    # Convert effect measure to proportion if it's in percentage
    mod_data$effect_measure <- mod_data$effect_measure / 100

    # Apply continuity correction and logit transformation
    epsilon <- 0.5 / min(mod_data$sample_size)
    mod_data$effect_measure <- pmin(pmax(mod_data$effect_measure, epsilon), 1 - epsilon)
    mod_data$logit_effect_measure <- log(mod_data$effect_measure / (1 - mod_data$effect_measure))
    # Calculate variance
    mod_data$var_logit_effect_measure <- 1 / (mod_data$sample_size * mod_data$effect_measure) + 
                                         1 / (mod_data$sample_size * (1 - mod_data$effect_measure))

    # Determine the formula based on moderator type
    formula <- as.formula(if (is.logical(mod_data$moderator)) {
                            "~ moderator - 1"
                          } else {
                            "~ moderator"
                          })

    # Perform meta-regression, wrapped in try() to handle errors
    res <- try(rma(yi = logit_effect_measure, 
                   vi = var_logit_effect_measure, 
                   mods = formula, 
                   data = mod_data, 
                   method = "DL"))

    print(summary(res))
            # Initialize variables
    pooled_estimate <- NA
    ci_lb <- NA
    ci_ub <- NA
    i2 <- NA
    tau2 <- NA
    p_value <- NA

    # Check for error in meta-regression
    if (!inherits(res, "try-error")) {
        # Extract results
      pooled_estimate <- res$b[-1]
      ci_lb <- res$ci.lb[-1]
      ci_ub <- res$ci.ub[-1]
      i2 <- res$I2
      tau2 <- res$tau2
      p_value <- res$pval[-1]
    } else {
      next  # Skip to next iteration if error occurred
    }    

    # Append results to dataframe
    temp_df <- data.frame(Moderator = col,
                          Pooled_Estimate = pooled_estimate, 
                          CI_Lower = ci_lb, 
                          CI_Upper = ci_ub, 
                          I2 = i2, 
                          Tau2 = tau2, 
                          P_Value = p_value)

    results <- rbind(results, temp_df)
  }

  # Convert results list to dataframe
  return(results)
}

combine_results <- function(results_df, effect_measure_name) {
  # Add a new column to the results dataframe indicating the effect measure
  results_df$Effect_Measure <- effect_measure_name

  # Return the modified dataframe
  return(results_df)
}

### 1 year patient survival
#### univariate meta-regression

In [161]:
# Select and rename columns
ps_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id', 
                       "1-year_patient_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx")]

names(ps_df_1) <- c('sample_size', 'study_id', '1_yr_ps', 'hivan', 'hcv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
ps_df_1$"viral_breakthrough" <- ps_df_1$"viral_suppress_post" < 1.0
ps_df_1$"hivan_major" <- ps_df_1$"hivan" > 0.5
ps_df_1$"ii_shift" <- (ps_df_1$'ii_pre' - ps_df_1$'ii_post') < 0
ps_df_1$"pi_avoid" <- (ps_df_1$'pi_pre' - ps_df_1$'pi_post') > 0

results_df <- meta_regression_function(ps_df_1, "study_id", "sample_size", "1_yr_ps")
print(results_df)
combined_results <- combine_results(results_df, "1_yr_ps")


Mixed-Effects Model (k = 16; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-13.3317    5.5826   32.6635   34.9812   34.6635   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.1788)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 14) = 5.5826, p-val = 0.9759

Test of Moderators (coefficient 2):
QM(df = 1) = 0.5501, p-val = 0.4583

Model Results:

           estimate      se     zval    pval    ci.lb   ci.ub      
intrcpt      2.8514  0.3408   8.3679  <.0001   2.1835  3.5193  *** 
moderator   -0.5532  0.7459  -0.7417  0.4583  -2.0151  0.9087      

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 19; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC    

#### Multivariate meta-regression

In [120]:
multi_mr_df <- ps_df_1[c("study_id", "sample_size", "1_yr_ps", "nnrti_pre", "viral_breakthrough", "hivan_major", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
print(multi_mr_df)
multi_mr_df$"1_yr_ps"<- multi_mr_df$"1_yr_ps" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"1_yr_ps" <- pmin(pmax(multi_mr_df$"1_yr_ps", epsilon), 1 - epsilon)
multi_mr_df$logit_1_yr_ps <- log(multi_mr_df$"1_yr_ps" / (1 - multi_mr_df$"1_yr_ps"))
# Calculate variance
multi_mr_df$var_logit_1_yr_ps <- 1 / (multi_mr_df$sample_size * multi_mr_df$"1_yr_ps") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"1_yr_ps"))
result <- rma(yi = logit_1_yr_ps, vi = var_logit_1_yr_ps, mods = ~ viral_breakthrough + hivan_major + ii_shift + pi_avoid - 1, data = multi_mr_df, method = "REML")
summary(result)

         study_id sample_size 1_yr_ps nnrti_pre viral_breakthrough hivan_major
3      Alfano2018          19    94.4    0.1579              FALSE       FALSE
6        Azar2017          13   100.0    0.1538              FALSE       FALSE
12     Durand2021          75   100.0    0.2667               TRUE       FALSE
35 Muthukumar2013          11   100.0    0.3636               TRUE       FALSE
37     Roland2008          18    94.0    0.5556               TRUE       FALSE
45     Touzot2010          27   100.0    0.1481               TRUE       FALSE
   ii_shift pi_avoid
3      TRUE     TRUE
6      TRUE     TRUE
12    FALSE    FALSE
35    FALSE    FALSE
37    FALSE    FALSE
45     TRUE     TRUE


Warning message:
“Redundant predictors dropped from the model.”



Mixed-Effects Model (k = 6; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -2.9641    5.9282   13.9282   10.3227   53.9282   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.7460)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00

Test for Residual Heterogeneity:
QE(df = 3) = 0.0859, p-val = 0.9935

Test of Moderators (coefficients 1:3):
QM(df = 3) = 66.2884, p-val < .0001

Model Results:

                         estimate      se    zval    pval    ci.lb   ci.ub      
viral_breakthroughFALSE    2.8411  1.3046  2.1779  0.0294   0.2843  5.3980    * 
viral_breakthroughTRUE     2.9819  0.4590  6.4965  <.0001   2.0822  3.8815  *** 
ii_shiftTRUE               0.0627  1.0316  0.0607  0.9516  -1.9593  2.0847      

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 3 year patient survival
#### univariate meta-regression


In [162]:
# Select and rename columns
ps_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id', 
                       "3-year_patient_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx")]

names(ps_df_3) <- c('sample_size', 'study_id', '3_yr_ps', 'hivan', 'hcv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
ps_df_3$"viral_breakthrough" <- ps_df_3$"viral_suppress_post" < 1.0
ps_df_3$"hivan_major" <- ps_df_3$"hivan" > 0.5
ps_df_3$"ii_shift" <- (ps_df_3$'ii_pre' - ps_df_3$'ii_post') < 0
ps_df_3$"pi_avoid" <- (ps_df_3$'pi_pre' - ps_df_3$'pi_post') > 0

results_df <- meta_regression_function(ps_df_3, "study_id", "sample_size", "3_yr_ps")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_ps"))


Mixed-Effects Model (k = 11; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-10.7129   16.4595   27.4258   28.6195   30.8544   

tau^2 (estimated amount of residual heterogeneity):     0.1778 (SE = 0.1876)
tau (square root of estimated tau^2 value):             0.4217
I^2 (residual heterogeneity / unaccounted variability): 52.17%
H^2 (unaccounted variability / sampling variability):   2.09
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 9) = 18.8149, p-val = 0.0268

Test of Moderators (coefficient 2):
QM(df = 1) = 0.0248, p-val = 0.8748

Model Results:

           estimate      se     zval    pval    ci.lb   ci.ub      
intrcpt      2.1842  0.3598   6.0704  <.0001   1.4790  2.8895  *** 
moderator   -0.1430  0.9072  -0.1576  0.8748  -1.9210  1.6351      

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 16; tau^2 estimator: DL)

  logLik  deviance       AIC   

#### Multivariate meta-regression

In [124]:
multi_mr_df <- ps_df_3[c("study_id", "sample_size", "3_yr_ps", "induction_use", "ii_pre", "viral_breakthrough", "hivan_major", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"3_yr_ps"<- multi_mr_df$"3_yr_ps" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"3_yr_ps" <- pmin(pmax(multi_mr_df$"3_yr_ps", epsilon), 1 - epsilon)
multi_mr_df$logit_3_yr_ps <- log(multi_mr_df$"3_yr_ps" / (1 - multi_mr_df$"3_yr_ps"))
# Calculate variance
multi_mr_df$var_logit_3_yr_ps <- 1 / (multi_mr_df$sample_size * multi_mr_df$"3_yr_ps") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"3_yr_ps"))
result <- rma(yi = logit_3_yr_ps, vi = var_logit_3_yr_ps, mods = ~ viral_breakthrough + hivan_major + ii_shift + pi_avoid, data = multi_mr_df, method = "REML")
summary(result)

Warning message:
“Redundant predictors dropped from the model.”



Mixed-Effects Model (k = 4; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -3.2619    6.5238   12.5238    8.6032   36.5238   

tau^2 (estimated amount of residual heterogeneity):     0.8394 (SE = 1.6446)
tau (square root of estimated tau^2 value):             0.9162
I^2 (residual heterogeneity / unaccounted variability): 51.31%
H^2 (unaccounted variability / sampling variability):   2.05
R^2 (amount of heterogeneity accounted for):            1.25%

Test for Residual Heterogeneity:
QE(df = 2) = 4.1189, p-val = 0.1275

Test of Moderators (coefficient 2):
QM(df = 1) = 0.6775, p-val = 0.4105

Model Results:

                        estimate      se    zval    pval    ci.lb   ci.ub    
intrcpt                   1.4245  0.8451  1.6856  0.0919  -0.2319  3.0810  . 
viral_breakthroughTRUE    1.0517  1.2778  0.8231  0.4105  -1.4526  3.5561    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 5 year and above patient survival
#### univariate meta-regression

In [163]:
# Select and rename columns
ps_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_patient_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(ps_df_5) <- c('sample_size', 'study_id', '5_yr_ps', 'hivan', 'hcv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
ps_df_5$"viral_breakthrough" <- ps_df_5$"viral_suppress_post" < 1.0
ps_df_5$"hivan_major" <- ps_df_5$"hivan" > 0.5
ps_df_5$"ii_shift" <- (ps_df_5$'ii_pre' - ps_df_5$'ii_post') < 0
ps_df_5$"pi_avoid" <- (ps_df_5$'pi_pre' - ps_df_5$'pi_post') > 0

results_df <- meta_regression_function(ps_df_5, "study_id", "sample_size", "5_yr_ps")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_ps"))


Mixed-Effects Model (k = 6; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -5.4318    9.0125   16.8636   16.2389   28.8636   

tau^2 (estimated amount of residual heterogeneity):     0.2682 (SE = 0.3478)
tau (square root of estimated tau^2 value):             0.5179
I^2 (residual heterogeneity / unaccounted variability): 56.76%
H^2 (unaccounted variability / sampling variability):   2.31
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 4) = 9.2500, p-val = 0.0551

Test of Moderators (coefficient 2):
QM(df = 1) = 0.0753, p-val = 0.7838

Model Results:

           estimate      se    zval    pval    ci.lb   ci.ub     
intrcpt      1.5577  0.5462  2.8517  0.0043   0.4871  2.6282  ** 
moderator    0.3320  1.2102  0.2743  0.7838  -2.0399  2.7039     

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 9; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC  

#### Multivariate meta-regression

In [126]:
multi_mr_df <- ps_df_5[c("study_id", "sample_size", "5_yr_ps", "il_2", "induction_use", "ii_pre", "ii_post", "nrti_post", "viral_breakthrough", "hivan_major", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"5_yr_ps"<- multi_mr_df$"5_yr_ps" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"5_yr_ps" <- pmin(pmax(multi_mr_df$"5_yr_ps", epsilon), 1 - epsilon)
multi_mr_df$logit_5_yr_ps <- log(multi_mr_df$"5_yr_ps" / (1 - multi_mr_df$"5_yr_ps"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_5_yr_ps <- 1 / (multi_mr_df$sample_size * multi_mr_df$"5_yr_ps") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"5_yr_ps"))
result <- rma(yi = logit_5_yr_ps, vi = var_logit_5_yr_ps, mods = ~ induction_use+viral_breakthrough+hivan_major+ii_shift, data = multi_mr_df, method = "REML")

summary(result)

    study_id sample_size 5_yr_ps il_2 induction_use ii_pre ii_post nrti_post
3 Alfano2018          19   0.708    1             1 0.4737  0.8421    0.7368
  viral_breakthrough hivan_major ii_shift pi_avoid logit_5_yr_ps
3              FALSE       FALSE     TRUE     TRUE     0.8856903


Warning message:
“Redundant predictors dropped from the model.”



Random-Effects Model (k = 1; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -0.0000    0.0000    4.0000      -Inf   16.0000   

tau^2 (estimated amount of total heterogeneity): 0
tau (square root of estimated tau^2 value):      0
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity:
Q(df = 0) = 0.0000, p-val = 1.0000

Model Results:

estimate      se    zval    pval    ci.lb   ci.ub    
  0.8857  0.5046  1.7554  0.0792  -0.1032  1.8746  . 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 1 year graft survival
#### univariate meta-regression

In [127]:
# Select and rename columns
gs_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "1-year_graft_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(gs_df_1) <- c('sample_size', 'study_id', '1_yr_gs', 'hivan', 'hcv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
gs_df_1$"viral_breakthrough" <- gs_df_1$"viral_suppress_post" < 1.0
gs_df_1$"hivan_major" <- gs_df_1$"hivan" > 0.5
gs_df_1$"ii_shift" <- (gs_df_1$'ii_pre' - gs_df_1$'ii_post') < 0
gs_df_1$"pi_avoid" <- (gs_df_1$'pi_pre' - gs_df_1$'pi_post') > 0

results_df <- meta_regression_function(gs_df_1, "study_id", "sample_size", "1_yr_gs")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "1_yr_gs"))

             Moderator Pooled_Estimate     CI_Lower   CI_Upper I2 Tau2
1                hivan     0.341084654 -1.075641295 1.75781060  0    0
2                  hcv    -0.647713746 -1.829648647 0.53422115  0    0
3                  cd4    -0.001474998 -0.005666066 0.00271607  0    0
4   viral_suppress_pre     2.048237694 -4.732691255 8.82916664  0    0
5  viral_suppress_post     0.727294568 -1.550151074 3.00474021  0    0
6                  atg     0.254650702 -0.734719639 1.24402104  0    0
7                 il_2     0.230477064 -0.855088920 1.31604305  0    0
8                 cd52    -0.174200994 -1.736800039 1.38839805  0    0
9        induction_use     0.293713665 -0.886564053 1.47399138  0    0
10              ii_pre     0.383276806 -0.720769106 1.48732272  0    0
11             ii_post     0.509263629 -0.559115419 1.57764268  0    0
12              pi_pre    -0.592726584 -2.283445038 1.09799187  0    0
13             pi_post    -1.027578800 -2.752441261 0.69728366  0    0
14    

#### Multivariate meta-regression

In [129]:
multi_mr_df <- gs_df_1[c("study_id", "sample_size", "1_yr_gs", "viral_breakthrough", "hivan_major", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"1_yr_gs"<- multi_mr_df$"1_yr_gs" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"1_yr_gs" <- pmin(pmax(multi_mr_df$"1_yr_gs", epsilon), 1 - epsilon)
multi_mr_df$logit_1_yr_gs <- log(multi_mr_df$"1_yr_gs" / (1 - multi_mr_df$"1_yr_gs"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_1_yr_gs <- 1 / (multi_mr_df$sample_size * multi_mr_df$"1_yr_gs") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"1_yr_gs"))
result <- rma(yi = logit_1_yr_gs, vi = var_logit_1_yr_gs, mods = ~ viral_breakthrough+hivan_major+ii_shift, data = multi_mr_df, method = "REML")

summary(result)

         study_id sample_size   1_yr_gs viral_breakthrough hivan_major ii_shift
3      Alfano2018          19 0.8950000              FALSE       FALSE     TRUE
6        Azar2017          13 0.9545455              FALSE       FALSE     TRUE
12     Durand2021          75 0.9171000               TRUE       FALSE    FALSE
35 Muthukumar2013          11 0.9100000               TRUE       FALSE    FALSE
37     Roland2008          18 0.8300000               TRUE       FALSE    FALSE
45     Touzot2010          27 0.9545455               TRUE       FALSE     TRUE
   pi_avoid logit_1_yr_gs
3      TRUE      2.142863
6      TRUE      3.044522
12    FALSE      2.403581
35    FALSE      2.313635
37    FALSE      1.585627
45     TRUE      3.044522


Warning message:
“Redundant predictors dropped from the model.”



Mixed-Effects Model (k = 6; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -2.8794    5.7589   13.7589   10.1533   53.7589   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.3635)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 3) = 1.5454, p-val = 0.6718

Test of Moderators (coefficients 2:3):
QM(df = 2) = 0.8150, p-val = 0.6653

Model Results:

                        estimate      se    zval    pval    ci.lb   ci.ub    
intrcpt                   1.4823  1.1784  1.2579  0.2084  -0.8273  3.7919    
viral_breakthroughTRUE    0.6852  1.1310  0.6058  0.5446  -1.5316  2.9020    
ii_shiftTRUE              0.8770  0.9813  0.8937  0.3715  -1.0463  2.8004    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.0

### 3 year graft survival
#### univariate meta-regression

In [164]:
# Select and rename columns
gs_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "3-year_graft_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(gs_df_3) <- c('sample_size', 'study_id', '3_yr_gs', 'hivan', 'hcv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
gs_df_3$"viral_breakthrough" <- gs_df_3$"viral_suppress_post" < 1.0
gs_df_3$"hivan_major" <- gs_df_3$"hivan" > 0.5
gs_df_3$"ii_shift" <- (gs_df_3$'ii_pre' - gs_df_3$'ii_post') < 0
gs_df_3$"pi_avoid" <- (gs_df_3$'pi_pre' - gs_df_3$'pi_post') > 0

results_df <- meta_regression_function(gs_df_3, "study_id", "sample_size", "3_yr_gs")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_gs"))


Mixed-Effects Model (k = 11; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -6.1329   11.7764   18.2658   19.4595   21.6944   

tau^2 (estimated amount of residual heterogeneity):     0.0628 (SE = 0.0821)
tau (square root of estimated tau^2 value):             0.2506
I^2 (residual heterogeneity / unaccounted variability): 40.53%
H^2 (unaccounted variability / sampling variability):   1.68
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 9) = 15.1346, p-val = 0.0873

Test of Moderators (coefficient 2):
QM(df = 1) = 0.0569, p-val = 0.8115

Model Results:

           estimate      se    zval    pval    ci.lb   ci.ub      
intrcpt      1.4389  0.2367  6.0795  <.0001   0.9750  1.9028  *** 
moderator    0.1533  0.6427  0.2385  0.8115  -1.1064  1.4130      

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 16; tau^2 estimator: DL)

  logLik  deviance       AIC      

#### Multivariate meta-regression

In [137]:
multi_mr_df <- gs_df_3[c("study_id", "sample_size", "3_yr_gs", "viral_suppress_post", "induction_use", "hivan_major", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"3_yr_gs"<- multi_mr_df$"3_yr_gs" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"3_yr_gs" <- pmin(pmax(multi_mr_df$"3_yr_gs", epsilon), 1 - epsilon)
multi_mr_df$logit_3_yr_gs <- log(multi_mr_df$"3_yr_gs" / (1 - multi_mr_df$"3_yr_gs"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_3_yr_gs <- 1 / (multi_mr_df$sample_size * multi_mr_df$"3_yr_gs") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"3_yr_gs"))
result <- rma(yi = logit_3_yr_gs, vi = var_logit_3_yr_gs, mods = ~ viral_suppress_post+induction_use, data = multi_mr_df, method = "REML")

summary(result)

         study_id sample_size   3_yr_gs viral_suppress_post induction_use
3      Alfano2018          19 0.7710000           1.0000000     1.0000000
6        Azar2017          13 0.9545455           1.0000000     1.0000000
35 Muthukumar2013          11 0.8100000           0.8181818     1.0000000
37     Roland2008          18 0.8300000           0.6111111     0.3888889
   hivan_major ii_shift pi_avoid logit_3_yr_gs
3        FALSE     TRUE     TRUE      1.213966
6        FALSE     TRUE     TRUE      3.044522
35       FALSE    FALSE    FALSE      1.450010
37       FALSE    FALSE    FALSE      1.585627



Mixed-Effects Model (k = 4; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -1.6770    3.3540   11.3540    3.3540   51.3540   

tau^2 (estimated amount of residual heterogeneity):     0.6400 (SE = 2.3695)
tau (square root of estimated tau^2 value):             0.8000
I^2 (residual heterogeneity / unaccounted variability): 38.20%
H^2 (unaccounted variability / sampling variability):   1.62
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 1) = 1.6180, p-val = 0.2034

Test of Moderators (coefficients 2:3):
QM(df = 2) = 0.0413, p-val = 0.9795

Model Results:

                     estimate      se     zval    pval     ci.lb    ci.ub    
intrcpt                0.9432  3.5193   0.2680  0.7887   -5.9545   7.8409    
viral_suppress_post    1.5202  7.5935   0.2002  0.8413  -13.3627  16.4032    
induction_use         -0.7370  4.4933  -0.1640  0.8697   -9.5437   8.0697    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0

### 5 year and above graft survival
#### univariate meta-regression

In [165]:
# Select and rename columns
gs_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_graft_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(gs_df_5) <- c('sample_size', 'study_id', '5_yr_gs', 'hivan', 'hcv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
gs_df_5$"viral_breakthrough" <- ps_df_5$"viral_suppress_post" < 1.0
gs_df_5$"hivan_major" <- ps_df_5$"hivan" > 0.5
gs_df_5$"ii_shift" <- (ps_df_5$'ii_pre' - ps_df_5$'ii_post') < 0
gs_df_5$"pi_avoid" <- (ps_df_5$'pi_pre' - ps_df_5$'pi_post') > 0

gesults_df <- meta_regression_function(gs_df_5, "study_id", "sample_size", "5_yr_gs")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_gs"))


Mixed-Effects Model (k = 6; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -0.8156    1.7673    7.6312    7.0065   19.6312   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.1001)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00
R^2 (amount of heterogeneity accounted for):            100.00%

Test for Residual Heterogeneity:
QE(df = 4) = 1.7673, p-val = 0.7785

Test of Moderators (coefficient 2):
QM(df = 1) = 5.9917, p-val = 0.0144

Model Results:

           estimate      se    zval    pval    ci.lb   ci.ub    
intrcpt      0.3923  0.2663  1.4735  0.1406  -0.1295  0.9142    
moderator    1.7576  0.7180  2.4478  0.0144   0.3503  3.1650  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 8; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   


### 1 year graft rejection
#### univariate meta-regression

In [166]:
# Select and rename columns
rj_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "1-year_cumulative_graft_rejection_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                     #   "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(rj_df_1) <- c('sample_size', 'study_id', '1_yr_rj', 'hivan', 'hcv', 
                     'cd4', 
                     # 'viral_suppress_pre', 
                     'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
rj_df_1$"viral_breakthrough" <- rj_df_1$"viral_suppress_post" < 1.0
rj_df_1$"hivan_major" <- rj_df_1$"hivan" > 0.5
rj_df_1$"ii_shift" <- (rj_df_1$'ii_pre' - rj_df_1$'ii_post') < 0
rj_df_1$"pi_avoid" <- (rj_df_1$'pi_pre' - rj_df_1$'pi_post') > 0
results_df <- meta_regression_function(rj_df_1, "study_id", "sample_size", "1_yr_rj")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "1_yr_rj"))


Mixed-Effects Model (k = 15; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-17.3125   30.1974   40.6251   42.7492   42.8069   

tau^2 (estimated amount of residual heterogeneity):     0.2164 (SE = 0.1630)
tau (square root of estimated tau^2 value):             0.4652
I^2 (residual heterogeneity / unaccounted variability): 59.46%
H^2 (unaccounted variability / sampling variability):   2.47
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 13) = 32.0664, p-val = 0.0023

Test of Moderators (coefficient 2):
QM(df = 1) = 0.6138, p-val = 0.4334

Model Results:

           estimate      se     zval    pval    ci.lb   ci.ub    
intrcpt     -0.6278  0.4308  -1.4572  0.1451  -1.4722  0.2166    
moderator   -0.7435  0.9490  -0.7835  0.4334  -2.6036  1.1166    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 15; tau^2 estimator: DL)

  logLik  deviance       AIC       B

### 3 year graft rejection
#### univariate meta-regression

In [167]:
# Select and rename columns
rj_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "3-year_cumulative_graft_rejection_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                      #  "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(rj_df_3) <- c('sample_size', 'study_id', '3_yr_rj', 'hivan', 'hcv', 
                     'cd4', 
                    #  'viral_suppress_pre', 
                     'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
rj_df_3$"viral_breakthrough" <- rj_df_3$"viral_suppress_post" < 1.0
rj_df_3$"hivan_major" <- rj_df_3$"hivan" > 0.5
rj_df_3$"ii_shift" <- (rj_df_3$'ii_pre' - rj_df_3$'ii_post') < 0
rj_df_3$"pi_avoid" <- (rj_df_3$'pi_pre' - rj_df_3$'pi_post') > 0
results_df <- meta_regression_function(rj_df_3, "study_id", "sample_size", "3_yr_rj")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_rj"))


Mixed-Effects Model (k = 8; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -8.5779   14.4781   23.1558   23.3941   29.1558   

tau^2 (estimated amount of residual heterogeneity):     0.2601 (SE = 0.2880)
tau (square root of estimated tau^2 value):             0.5100
I^2 (residual heterogeneity / unaccounted variability): 56.28%
H^2 (unaccounted variability / sampling variability):   2.29
R^2 (amount of heterogeneity accounted for):            32.44%

Test for Residual Heterogeneity:
QE(df = 6) = 13.7232, p-val = 0.0329

Test of Moderators (coefficient 2):
QM(df = 1) = 2.7341, p-val = 0.0982

Model Results:

           estimate      se     zval    pval    ci.lb   ci.ub    
intrcpt     -0.0683  0.4867  -0.1404  0.8883  -1.0223  0.8856    
moderator   -1.6045  0.9704  -1.6535  0.0982  -3.5064  0.2974  . 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 6; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC

### 5 year and above graft rejection
#### univariate meta-regression

In [168]:
# Select and rename columns
rj_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_cumulative_graft_rejection_(%)_hiv+", 
                       "hivan_pre-transplant",
                     #   "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(rj_df_5) <- c('sample_size', 'study_id', '5_yr_rj', 'hivan', 
                     # 'hcv', 
                     'cd4', 
                     'viral_suppress_pre', 
                     'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
rj_df_5$"viral_breakthrough" <- rj_df_5$"viral_suppress_post" < 1.0
# rj_df_5$"hivan_major" <- rj_df_5$"hivan" > 0.5
rj_df_5$"ii_shift" <- (rj_df_5$'ii_pre' - rj_df_5$'ii_post') < 0
rj_df_5$"pi_avoid" <- (rj_df_5$'pi_pre' - rj_df_5$'pi_post') > 0
print(rj_df_5)
results_df <- meta_regression_function(rj_df_5, "study_id", "sample_size", "5_yr_rj")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_rj"))

   sample_size          study_id 5_yr_rj      hivan    cd4 viral_suppress_pre
1           47        Abbott2004      NA         NA     NA                 NA
2           24      Ailioaie2017      NA 0.54000000 377.00          0.8800000
3           19        Alfano2018   48.80 0.10526316 407.00          1.0000000
4           22        Alfano2020      NA         NA     NA                 NA
5          605      Apewokin2018   27.50         NA     NA                 NA
6           13          Azar2017      NA 0.23076923 536.00          1.0000000
7          104         Boyle2017      NA         NA 452.61                 NA
8           42         Boyle2020      NA 0.19047619     NA                 NA
9           22       Camargo2019      NA 0.86363636 453.00          1.0000000
10          13      Camargo2019a      NA 0.53846154 525.00          1.0000000
11          17       Delaney1992   58.82         NA     NA                 NA
12          75        Durand2021      NA 0.29333333 528.00      

### 1 year infection
#### univariate meta-regression
- the infection outcome present as incidence, function need to be modified from logit transformation to log transformation, and assumed Poisson distribution

In [169]:
meta_regression_function_2 <- function(df, study_id_col, sample_size_col, effect_measure_col) {
  results <- list()

  # List of columns to perform meta-regression on
  columns_to_analyze <- setdiff(names(df), c(study_id_col, sample_size_col, effect_measure_col))

  for (col in columns_to_analyze) {
    # Prepare data for meta-regression
    mod_data <- df[, c(sample_size_col, effect_measure_col, col)]
    mod_data <- na.omit(mod_data)
    names(mod_data) <- c("sample_size", "effect_measure", "moderator")

    if (dim(mod_data)[1] < 2) {
      next
    }

    # Apply continuity correction and logit transformation
    epsilon <- 0.5 / min(mod_data$sample_size)
    mod_data$log_effect_measure <- log(mod_data$effect_measure)
    # Calculate variance
    # assuming Poisson distribution
    mod_data$var_log_effect_measure <- 1 / (mod_data$effect_measure * mod_data$sample_size)

    # Determine the formula based on moderator type
    formula <- as.formula(if (is.logical(mod_data$moderator)) {
                            "~ moderator - 1"
                          } else {
                            "~ moderator"
                          })

    # Perform meta-regression, wrapped in try() to handle errors
    res <- try(rma(yi = logit_effect_measure, 
                   vi = var_logit_effect_measure, 
                   mods = formula, 
                   data = mod_data, 
                   method = "DL"))

    print(summary(res))
            # Initialize variables
    pooled_estimate <- NA
    ci_lb <- NA
    ci_ub <- NA
    i2 <- NA
    tau2 <- NA
    p_value <- NA

    # Check for error in meta-regression
    if (!inherits(res, "try-error")) {
        # Extract results
      pooled_estimate <- res$b[-1]
      ci_lb <- res$ci.lb[-1]
      ci_ub <- res$ci.ub[-1]
      i2 <- res$I2
      tau2 <- res$tau2
      p_value <- res$pval[-1]
    } else {
      next  # Skip to next iteration if error occurred
    }    

    # Append results to dataframe
    temp_df <- data.frame(Moderator = col,
                          Pooled_Estimate = pooled_estimate, 
                          CI_Lower = ci_lb, 
                          CI_Upper = ci_ub, 
                          I2 = i2, 
                          Tau2 = tau2, 
                          P_Value = p_value)

    results <- rbind(results, temp_df)
  }

  # Convert results list to dataframe
  return(results)
}

In [204]:
# Select and rename columns
if_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "1-year_infection_incidence(100_pt-yr)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                      #  "viral_suppression_(<=_50copies/mm3)_pre-transplant"
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                      #  "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                      #  "induction_total"
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(if_df_1) <- c('sample_size', 'study_id', '1_yr_if', 
                    'hivan', 
                    'hcv',
                    'cd4', 
                    # 'viral_suppress_pre'
                    'viral_suppress_post',
                    # 'atg',
                    'il_2',
                    'cd52',
                    # 'induction_use'
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post',
                    'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post'
                    )
if_df_1$"viral_breakthrough" <- if_df_1$"viral_suppress_post" < 1.0
if_df_1$"hivan_major" <- if_df_1$"hivan" > 0.5
if_df_1$"ii_shift" <- (if_df_1$'ii_pre' - if_df_1$'ii_post') < 0
if_df_1$"pi_avoid" <- (if_df_1$'pi_pre' - if_df_1$'pi_post') > 0

results_df <- meta_regression_function(if_df_1, "study_id", "sample_size", "1_yr_if")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "1_yr_if"))


Mixed-Effects Model (k = 5; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -8.1635   13.9649   22.3269   21.1553   46.3269   

tau^2 (estimated amount of residual heterogeneity):     2.4906 (SE = 2.4071)
tau (square root of estimated tau^2 value):             1.5782
I^2 (residual heterogeneity / unaccounted variability): 91.34%
H^2 (unaccounted variability / sampling variability):   11.55
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 3) = 34.6501, p-val < .0001

Test of Moderators (coefficient 2):
QM(df = 1) = 0.9653, p-val = 0.3259

Model Results:

           estimate      se     zval    pval     ci.lb   ci.ub    
intrcpt      2.2788  1.9033   1.1973  0.2312   -1.4516  6.0092    
moderator   -5.0444  5.1343  -0.9825  0.3259  -15.1074  5.0186    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 3; tau^2 estimator: DL)

  logLik  deviance       AIC       B

### 3 year infection
#### univariate meta-regression

In [205]:
# Select and rename columns
if_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "3-year_infection_incidence(100_pt-yr)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(if_df_3) <- c('sample_size', 'study_id', '3_yr_if', 
                    'hivan', 
                    'hcv',
                    'cd4', 
                    'viral_suppress_pre',
                    'viral_suppress_post',
                    'atg',
                    'il_2',
                    'cd52',
                    'induction_use',
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post',
                    'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post'
                    )
if_df_3$"viral_breakthrough" <- if_df_3$"viral_suppress_post" < 1.0
if_df_3$"hivan_major" <- if_df_3$"hivan" > 0.5
if_df_3$"ii_shift" <- (if_df_3$'ii_pre' - if_df_3$'ii_post') < 0
if_df_3$"pi_avoid" <- (if_df_3$'pi_pre' - if_df_3$'pi_post') > 0

results_df <- meta_regression_function(if_df_3, "study_id", "sample_size", "3_yr_if")
print(results_df)
# combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_if"))

list()


### 5 year and above infection
#### univariate meta-regression

In [207]:
# Select and rename columns
if_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_graft_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(if_df_5) <- c('sample_size', 'study_id', '5_yr_if', 
                    'hivan', 
                    'hcv',
                    'cd4', 
                    'viral_suppress_pre',
                    'viral_suppress_post',
                    'atg',
                    'il_2',
                    'cd52',
                    'induction_use',
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post',
                    'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post'
                    )
if_df_5$"viral_breakthrough" <- if_df_5$"viral_suppress_post" < 1.0
if_df_5$"hivan_major" <- if_df_5$"hivan" > 0.5
if_df_5$"ii_shift" <- (if_df_5$'ii_pre' - if_df_5$'ii_post') < 0
if_df_5$"pi_avoid" <- (if_df_5$'pi_pre' - if_df_5$'pi_post') > 0

results_df <- meta_regression_function(if_df_5, "study_id", "sample_size", "5_yr_if")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_if"))


Mixed-Effects Model (k = 6; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -0.8156    1.7673    7.6312    7.0065   19.6312   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.1001)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00
R^2 (amount of heterogeneity accounted for):            100.00%

Test for Residual Heterogeneity:
QE(df = 4) = 1.7673, p-val = 0.7785

Test of Moderators (coefficient 2):
QM(df = 1) = 5.9917, p-val = 0.0144

Model Results:

           estimate      se    zval    pval    ci.lb   ci.ub    
intrcpt      0.3923  0.2663  1.4735  0.1406  -0.1295  0.9142    
moderator    1.7576  0.7180  2.4478  0.0144   0.3503  3.1650  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Mixed-Effects Model (k = 8; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   


In [213]:
# print(combined_results)
selected_rows <- combined_results[combined_results$P_Value < 0.05, ]
# Group by Moderator and arrange within each group
grouped_sorted_df <- selected_rows %>%
                     group_by(Moderator) %>%
                     arrange(Moderator)
library(knitr)
kable(grouped_sorted_df, format = "markdown", caption = "Selected Rows with P-Value < 0.05")




Table: Selected Rows with P-Value < 0.05

|Moderator          | Pooled_Estimate|    CI_Lower|    CI_Upper|        I2|      Tau2|   P_Value|Effect_Measure |
|:------------------|---------------:|-----------:|-----------:|---------:|---------:|---------:|:--------------|
|atg                |      -1.5139755|  -2.4168124|  -0.6111386|  0.000000| 0.0000000| 0.0010138|3_yr_rj        |
|atg                |      -3.9647835|  -6.4159191|  -1.5136480| 59.288049| 0.0640878| 0.0015228|5_yr_rj        |
|cd52               |     -36.6876939| -49.8720542| -23.5033337|  0.000000| 0.0000000| 0.0000000|1_yr_rj        |
|hivan              |       1.7576279|   0.3502806|   3.1649751|  0.000000| 0.0000000| 0.0143736|5_yr_if        |
|hivan_major        |       2.4777821|   1.9935647|   2.9619995|  0.000000| 0.0000000| 0.0000000|1_yr_ps        |
|hivan_major        |       2.0983707|   1.4325067|   2.7642347| 51.932684| 0.1529968| 0.0000000|3_yr_ps        |
|hivan_major        |       1.8223433|   0.9